##### Copyright 2018 The TensorFlow Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License"); { display-mode: "form" }
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/tensorflow_text/intro"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/text/blob/master/examples/intro.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/text/blob/master/examples/intro.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/text/examples/intro.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

# TF.Text

## Introduction

TensorFlow Text provides a collection of text related classes and ops ready to use with TensorFlow 2.0. The library can perform the preprocessing regularly required by text-based models, and includes other features useful for sequence modeling not provided by core TensorFlow.

The benefit of using these ops in your text preprocessing is that they are done in the TensorFlow graph. You do not need to worry about tokenization in training being different than the tokenization at inference, or managing preprocessing scripts.  
  
소개
TensorFlow Text는 TensorFlow 2.0에서 사용할 준비가 된 텍스트 관련 클래스 및 작업 모음을 제공합니다. 라이브러리는 텍스트 기반 모델에 필요한 사전 처리를 정기적으로 수행 할 수 있으며 핵심 TensorFlow에서 제공하지 않는 시퀀스 모델링에 유용한 기타 기능을 포함합니다.

텍스트 전처리에서 이러한 작업을 사용하는 이점은 TensorFlow 그래프에서 수행된다는 것입니다. 교육에서 토큰 화가 추론 또는 전처리 스크립트 관리에서 토큰 화와 다른 것에 대해 걱정할 필요가 없습니다.

## Eager Execution

TensorFlow Text requires TensorFlow 2.0, and is fully compatible with eager mode and graph mode.

---

Note: On rare occassions, this import may fail looking for the TF library. Please reset the runtime and rerun the pip install below.

In [ ]:
!pip install tensorflow-text

In [ ]:
import tensorflow as tf
import tensorflow_text as text

## Unicode

Most ops expect that the strings are in UTF-8. If you're using a different encoding, you can use the core tensorflow transcode op to transcode into UTF-8. You can also use the same op to coerce your string to structurally valid UTF-8 if your input could be invalid.  
  
유니 코드
대부분의 작업은 문자열이 UTF-8로되어 있다고 예상합니다. 다른 인코딩을 사용하는 경우 핵심 tensorflow 트랜스 코딩 작업을 사용하여 UTF-8로 트랜스 코딩 할 수 있습니다. 입력이 유효하지 않은 경우 동일한 연산을 사용하여 문자열을 구조적으로 유효한 UTF-8로 강제 변환 할 수도 있습니다.

In [ ]:
docs = tf.constant([u'Everything not saved will be lost.'.encode('UTF-16-BE'), u'Sad☹'.encode('UTF-16-BE')])
utf8_docs = tf.strings.unicode_transcode(docs, input_encoding='UTF-16-BE', output_encoding='UTF-8')

## Tokenization

Tokenization is the process of breaking up a string into tokens. Commonly, these tokens are words, numbers, and/or punctuation.

The main interfaces are `Tokenizer` and `TokenizerWithOffsets` which each have a single method `tokenize` and `tokenize_with_offsets` respectively. There are multiple tokenizers available now. Each of these implement `TokenizerWithOffsets` (which extends `Tokenizer`) which includes an option for getting byte offsets into the original string. This allows the caller to know the bytes in the original string the token was created from.

All of the tokenizers return RaggedTensors with the inner-most dimension of tokens mapping to the original individual strings. As a result, the resulting shape's rank is increased by one. Please review the ragged tensor guide if you are unfamiliar with them. [ragged_tensors](https://www.tensorflow.org/guide/ragged_tensors)

### WhitespaceTokenizer

This is a basic tokenizer that splits UTF-8 strings on ICU defined whitespace characters (eg. space, tab, new line).  
  
토큰 화
토큰 화는 문자열을 토큰으로 나누는 프로세스입니다. 일반적으로 이러한 토큰은 단어, 숫자 및 / 또는 구두점입니다.

주요 인터페이스는 각각 단일 메서드 tokenize 및 tokenize_with_offsets 가있는 Tokenizer 및 TokenizerWithOffsets 입니다. 현재 사용 가능한 여러 토크 나이저가 있습니다. 이러한 각각의 구현 TokenizerWithOffsets (확장하는 Tokenizer ) 원래의 문자열로 바이트 오프셋을 얻기위한 옵션이 포함되어있다. 이를 통해 호출자는 토큰이 생성 된 원래 문자열의 바이트를 알 수 있습니다.

모든 토크 나이 저는 원래 개별 문자열에 매핑되는 토큰의 가장 안쪽 차원이있는 RaggedTensors를 반환합니다. 결과적으로 결과 모양의 순위가 1 씩 증가합니다. 익숙하지 않은 경우 비정형 텐서 가이드를 검토하십시오. https://www.tensorflow.org/guide/ragged_tensors


In [ ]:
tokenizer = text.WhitespaceTokenizer()
tokens = tokenizer.tokenize(['everything not saved will be lost.', u'Sad☹'.encode('UTF-8')])
print(tokens.to_list())

### UnicodeScriptTokenizer

This tokenizer splits UTF-8 strings based on Unicode script boundaries. The script codes used correspond to International Components for Unicode (ICU) UScriptCode values. See: http://icu-project.org/apiref/icu4c/uscript_8h.html

In practice, this is similar to the `WhitespaceTokenizer` with the most apparent difference being that it will split punctuation (USCRIPT_COMMON) from language texts (eg. USCRIPT_LATIN, USCRIPT_CYRILLIC, etc) while also separating language texts from each other.  
  
UnicodeScriptTokenizer
이 토크 나이 저는 유니 코드 스크립트 경계를 기반으로 UTF-8 문자열을 분할합니다. 사용되는 스크립트 코드는 ICU (International Components for Unicode) UScriptCode 값에 해당합니다. 참조 : http://icu-project.org/apiref/icu4c/uscript_8h.html

실제로 이것은 WhitespaceTokenizer 와 유사하지만 언어 텍스트 (예 : USCRIPT_LATIN, USCRIPT_CYRILLIC 등)에서 구두점 (USCRIPT_COMMON)을 분리하는 동시에 언어 텍스트를 서로 분리한다는 점이 가장 뚜렷합니다.

In [ ]:
tokenizer = text.UnicodeScriptTokenizer()
tokens = tokenizer.tokenize(['everything not saved will be lost.', u'Sad☹'.encode('UTF-8')])
print(tokens.to_list())

### Unicode split

When tokenizing languages without whitespace to segment words, it is common to just split by character, which can be accomplished using the [unicode_split](https://www.tensorflow.org/api_docs/python/tf/strings/unicode_split) op found in core.

In [ ]:
tokens = tf.strings.unicode_split([u"仅今年前".encode('UTF-8')], 'UTF-8')
print(tokens.to_list())

### Offsets

When tokenizing strings, it is often desired to know where in the original string the token originated from. For this reason, each tokenizer which implements `TokenizerWithOffsets` has a *tokenize_with_offsets* method that will return the byte offsets along with the tokens. The start_offsets lists the bytes in the original string each token starts at, and the end_offsets lists the bytes immediately after the point where each token ends.  Note: the start offsets are inclusive and the end offsets are exclusive.  
  
오프셋
문자열을 토큰화할 때 원래 문자열에서 토큰의 출처를 알고 싶은 경우가 종종 있습니다. 이러한 이유로 TokenizerWithOffsets 를 구현하는 각 토크 나이저에는 토큰과 함께 바이트 오프셋을 반환하는 tokenize_with_offsets 메서드가 있습니다. offset_starts는 각 토큰이 시작하는 원래 문자열의 바이트를 나열하고 offset_limits는 각 토큰이 끝나는 바이트를 나열합니다.

In [ ]:
tokenizer = text.UnicodeScriptTokenizer()
(tokens, start_offsets, end_offsets) = tokenizer.tokenize_with_offsets(['everything not saved will be lost.', u'Sad☹'.encode('UTF-8')])
print(tokens.to_list())
print(start_offsets.to_list())
print(end_offsets.to_list())

### TF.Data Example

Tokenizers work as expected with the tf.data API. A simple example is provided below.

In [ ]:
docs = tf.data.Dataset.from_tensor_slices([['Never tell me the odds.'], ["It's a trap!"]])
tokenizer = text.WhitespaceTokenizer()
tokenized_docs = docs.map(lambda x: tokenizer.tokenize(x))
iterator = iter(tokenized_docs)
print(next(iterator).to_list())
print(next(iterator).to_list())

## Other Text Ops

TF.Text packages other useful preprocessing ops. We will review a couple below.

### Wordshape

A common feature used in some natural language understanding models is to see if the text string has a certain property. For example, a sentence breaking model might contain features which check for word capitalization or if a punctuation character is at the end of a string.

Wordshape defines a variety of useful regular expression based helper functions for matching various relevant patterns in your input text. Here are a few examples.  
  
기타 텍스트 작업
TF.Text는 다른 유용한 전처리 작업을 패키징합니다. 아래에서 몇 가지를 검토하겠습니다.

워드 셰이프
일부 자연어 이해 모델에서 사용되는 일반적인 기능은 텍스트 문자열에 특정 속성이 있는지 확인하는 것입니다. 예를 들어, 문장 분리 모델에는 단어 대문자 사용 또는 구두점 문자가 문자열 끝에 있는지 확인하는 기능이 포함될 수 있습니다.

Wordshape는 입력 텍스트에서 다양한 관련 패턴을 일치시키기 위해 유용한 정규식 기반 도우미 함수를 다양하게 정의합니다. 다음은 몇 가지 예입니다.

In [ ]:
tokenizer = text.WhitespaceTokenizer()
tokens = tokenizer.tokenize(['Everything not saved will be lost.', u'Sad☹'.encode('UTF-8')])

# Is capitalized?
f1 = text.wordshape(tokens, text.WordShape.HAS_TITLE_CASE)
# Are all letters uppercased?
f2 = text.wordshape(tokens, text.WordShape.IS_UPPERCASE)
# Does the token contain punctuation?
f3 = text.wordshape(tokens, text.WordShape.HAS_SOME_PUNCT_OR_SYMBOL)
# Is the token a number?
f4 = text.wordshape(tokens, text.WordShape.IS_NUMERIC_VALUE)

print(f1.to_list())
print(f2.to_list())
print(f3.to_list())
print(f4.to_list())

### N-grams & Sliding Window

N-grams are sequential words given a sliding window size of *n*. When combining the tokens, there are three reduction mechanisms supported. For text, you would want to use `Reduction.STRING_JOIN` which appends the strings to each other. The default separator character is a space, but this can be changed with the string_separater argument.

The other two reduction methods are most often used with numerical values, and these are `Reduction.SUM` and `Reduction.MEAN`.  
  
N- 그램 및 슬라이딩 윈도우
N- 그램은 슬라이딩 윈도우 크기가 n 인 경우 순차적 인 단어입니다. 토큰을 결합 할 때 지원되는 세 가지 감소 메커니즘이 있습니다. 텍스트의 경우 문자열을 서로 추가하는 Reduction.STRING_JOIN 을 사용할 수 있습니다. 기본 구분 문자는 공백이지만 string_separater 인수로 변경할 수 있습니다.

다른 두 가지 감소 방법은 숫자 값과 함께 가장 자주 사용되며 Reduction.SUM 및 Reduction.MEAN 입니다.

In [ ]:
tokenizer = text.WhitespaceTokenizer()
tokens = tokenizer.tokenize(['Everything not saved will be lost.', u'Sad☹'.encode('UTF-8')])

# Ngrams, in this case bi-gram (n = 2)
bigrams = text.ngrams(tokens, 2, reduction_type=text.Reduction.STRING_JOIN)

print(bigrams.to_list())